# Chapter 4, Exercise 1: Implement your own Learner

> Create your own implmentation of Learner from scratch, based on the training loop shown in this chapter.

As a reminder, the loop is:

- Init
- Predict
- Loss 
- Gradient
- Step
- Stop

Let's start with the boilerplate:

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.set_printoptions(edgeitems=2)
torch.manual_seed(42) # Life, the Universe, and Everything

I'll use the signature from the book; however, for now I'm going to leave out metrics.  I may come back to this later.

In [2]:
class MyLearner():
    
    def __init__(self, loader, model, opt_func, loss_func, metrics=None):
        self.loader = loader
        self.model = model
        self.opt_func = opt_func
        self.loss_func = loss_func
        self.metrics = metrics
        
    def fit(self, iterations=10):
        '''Fit method 
        '''
        pass

I don't know that I'll do every convenience function they have -- `learn.recorder.values()` for example -- but I'll see what I can do.

Let's create our model.  [Weights are  initialized for us](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html#torch.nn.Linear).

In [3]:
simple_net = nn.Sequential(
    nn.Linear(in_features=28*28, out_features=30),
    nn.ReLU(),
    nn.Linear(30, 1)
)

Next, the data loader...which I guess means we'll need some data.  We'll use the FastAI 3/7 image set.

In [4]:
from fastai.vision.all import *
path = untar_data(URLs.MNIST_SAMPLE)
path.ls()

(#3) [Path('/home/aardvark/.fastai/data/mnist_sample/labels.csv'),Path('/home/aardvark/.fastai/data/mnist_sample/valid'),Path('/home/aardvark/.fastai/data/mnist_sample/train')]

Let's load those into tensors:

In [5]:
training_3 = torch.stack([tensor(Image.open(o)) for o in (path /'train/3').ls().sorted()]).float() / 255.0
training_7 = torch.stack([tensor(Image.open(o)) for o in (path /'train/7').ls().sorted()]).float() / 255.0
len(training_3), len(training_7)

(6131, 6265)

In [6]:
all_training_data = torch.cat([training_3, training_7])
all_training_data.shape

torch.Size([12396, 28, 28])

Let's take a moment to remember how to change the shape to match our model input:

In [7]:
training_3[0].view(-1).shape, all_training_data.view(-1, 28*28).shape

(torch.Size([784]), torch.Size([12396, 784]))

Time for some labels.

In [8]:
labels_3 = torch.ones([len(training_3)])
labels_7 = torch.zeros([len(training_7)])
all_training_labels = torch.cat([labels_3, labels_7])
all_training_labels.shape

torch.Size([12396])

In [9]:
len(labels_3)

6131

Now time for the loader:

In [10]:
class NoMoreData(Exception):
    '''Nothing left, yo
    '''

class MyLoader():
    def __init__(self, training_data, labels, batch_size=5): # batch size picked randomly
        assert len(training_data) == len(labels)
        self.length = len(training_data)
        self.training_data = training_data
        self.labels = labels
        self.batch_size = batch_size
        self.counter = 0
        
    def next(self):
        '''Yield next batch of data.
        '''
        if self.counter == -1:
            # Using this as a signal we're at the end of our rope
            raise NoMoreData
        if (self.length - self.counter > self.batch_size):
            start = self.counter
            end = self.counter + self.batch_size
            self.counter += self.batch_size
            training_data_to_return =  self.training_data[start:end]
            training_labels_to_return = self.labels[start:end]
        elif (self.length - self.counter < self.batch_size):
            start = self.counter
            self.counter = -1
            training_data_to_return = self.training_data[start:]
            training_labels_to_return = self.labels[start:]
        return (training_data_to_return, training_labels_to_return)
    
    def reset(self):
        '''Reset counter so we can get more data
        '''
        self.counter = 0

It would be interesting to try and make this more like C library calls (not sure what the usual practice is there, but I'll bet you I'm not following it 🤣).  It would also be interesting to make this a Python yielder (oh, there's a better term for that...).  But for now, I'll stick with this.

In [11]:
my_training_loader = MyLoader(training_data=all_training_data.view(-1, 28*28), labels=all_training_labels)

In [12]:
a, b = my_training_loader.next()
a.shape, b.shape, my_training_loader.counter

(torch.Size([5, 784]), torch.Size([5]), 5)

In [13]:
my_training_loader.reset()
my_training_loader.counter

0

Aw crap...so:  next up would be optimizer.  I want to use the PyTorch SGD optimizer...but this is pretty closely tied with the rest of the Pytorch model API:

> To construct an Optimizer you have to give it an iterable containing the parameters (all should be Variable s) to optimize. Then, you can specify optimizer-specific options such as the learning rate, weight decay, etc.

```
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam([var1, var2], lr=0.0001)
```

...but wait: we *have* got a PyTorch model :doh:.  Let's use that.

In [14]:
my_optimizer = optim.SGD(simple_net.parameters(), lr=0.01, momentum=0.9)

As for loss function, let's just go simple and use mse.   We'll save something fancier for when we get into MNIST.

In [15]:
def my_loss(predicted, actual):
    return (F.mean(predicted - actual))**2

Now it's time to try some training!

In [16]:
my_loader = MyLoader

# TODO

- Run the loop.